In [1]:
import os
os.chdir('../py')

import gc

import pandas as pd
import numpy as np
import feather

from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams['font.family'] = 'Ricty Diminised'
sns.set(style="whitegrid", font='Ricty Diminised', palette="muted", color_codes=True, rc={'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

import japanize_matplotlib

from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import LabelEncoder

# my modules
import EDA
import utils

from scipy.stats import ks_2samp

import datetime

import super_aggre

In [2]:
train = pd.read_pickle('../input/pickle/train_df.pkl')

In [3]:
test = pd.read_pickle('../input/pickle/test_df.pkl')

In [4]:
train.columns.tolist()

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V

In [9]:
super_aggre.auto_agg(
    data=train,
    group='DT_M',
    agg_cols=['D9'],
    agg_funcs=['min', 'mean', 'max', 'std']
)

,D9_min,D9_mean,D9_max,D9_std
DT_M,,,,
12,621,744884.035370,947967,386744.663311
13,621,862366.563655,947967,270391.275898
14,621,854343.707153,947967,281456.375894
15,621,856699.793146,947967,278301.666325
16,621,860277.258179,947967,273365.931477
17,621,836779.877393,947967,303551.805790


In [13]:
train.columns.tolist()

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V

In [17]:
[col for col in train.columns if 'D' in col]

['TransactionID',
 'TransactionDT',
 'ProductCD',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'DT',
 'DT_M',
 'DT_W',
 'DT_D',
 'DT_hour',
 'DT_day_week',
 'DT_day_month',
 'DT_M_total',
 'DT_W_total',
 'DT_D_total',
 'card3_DT_D_hour_dist',
 'card3_DT_W_week_day_dist',
 'card3_DT_M_month_day_dist',
 'card3_DT_D_hour_dist_best',
 'card3_DT_W_week_day_dist_best',
 'card3_DT_M_month_day_dist_best',
 'card5_DT_D_hour_dist',
 'card5_DT_W_week_day_dist',
 'card5_DT_M_month_day_dist',
 'card5_DT_D_hour_dist_best',
 'card5_DT_W_week_day_dist_best',
 'card5_DT_M_month_day_dist_best',
 'bank_type_DT_D_hour_dist',
 'bank_type_DT_W_week_day_dist',
 'bank_type_DT_M_month_day_dist',
 'bank_type_DT_D_hour_dist_best',
 'bank_type_DT_W_week_day_dist_best',
 'bank_type_DT_M_month_day_dist_best',
 'bank_type_DT_M',
 'bank_type_DT_W',
 'bank_type_DT_D',
 'uid_D1_mean',
 'uid_D1_std',
 'uid2_D1_mean',
 'uid2_D1_std',
 'uid3_D1_mean',
 'ui

In [12]:
plt.figure(figsize=(8, 8))
sns.heatmap(train[['DT_M', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15']].corr()['DT_M'], cmap='RdBu', annot=True, vmin=-1, vmax=1)

IndexError: Inconsistent shape between the condition and the input (got (16, 1) and (16,))

<Figure size 576x576 with 0 Axes>

In [16]:
super_aggre.auto_agg(
    data=train,
    group=['DT_M', 'D1', 'D2', 'D3', 'D4', 'D5', '', 'D9'],
    agg_cols=['isFraud'],
    agg_funcs=['mean', 'std']
)

isFraud_mean  isFraud_std
DT_M D9                               
12   621         0.194690     0.397726
     799         0.032680     0.178381
     818         0.061856     0.242145
     1225        0.081967     0.274879
     1528        0.062500     0.242645
     2213        0.111888     0.315597
     2624        0.051020     0.220321
     3752        0.088608     0.284357
     4720        0.015094     0.122005
     5546        0.102733     0.303753
     6842        0.056492     0.230968
     7425        0.027891     0.164717
     7567        0.035821     0.185913
     7869        0.053123     0.224359
     8252        0.065392     0.247292
     8590        0.058612     0.234968
     8795        0.024931     0.155957
     9197        0.049180     0.216308
     9662        0.049470     0.216902
     10090       0.029279     0.168626
     10180       0.043478     0.203976
     10212       0.028480     0.166377
     10238       0.028842     0.167400
     10499       0.033476     0.179916
     947967      0.020401     0.141369
13   621         0.600000     0.497050
     799         0.280702     0.453336
     818         0.458333     0.503534
     1225        0.414634     0.495691
     1528        0.274194     0.449749
...                   ...          ...
16   10180       0.128310     0.334775
     10212       0.150362     0.357750
     10238       0.096339     0.295340
     10499       0.107505     0.310069
     947967      0.024008     0.153075
17   621         0.081081     0.276725
     799         0.200000     0.408248
     818         0.024793     0.156141
     1225        0.354839     0.482370
     1528        0.187500     0.391426
     2213        0.309278     0.464597
     2624        0.080986     0.273295
     3752        0.320000     0.467815
     4720        0.058182     0.234300
     5546        0.217262     0.412997
     6842        0.138425     0.345758
     7425        0.070890     0.256835
     7567        0.172840     0.378498
     7869        0.127413     0.333758
     8252        0.106287     0.308436
     8590        0.134328     0.341260
     8795        0.052632     0.223448
     9197        0.138177     0.345331
     9662        0.159468     0.366417
     10090       0.106061     0.308127
     10180       0.106349     0.308529
     10212       0.134349     0.341263
     10238       0.061770     0.240938
     10499       0.105735     0.307774
     947967      0.023477     0.151413

[150 rows x 2 columns]

In [11]:
train.groupby(['DT_M', 'D9'], as_index=False)['isFraud'].mean()

,DT_M,D9,isFraud
0,12,621,0.194690
1,12,799,0.032680
2,12,818,0.061856
3,12,1225,0.081967
4,12,1528,0.062500
5,12,2213,0.111888
6,12,2624,0.051020
7,12,3752,0.088608
8,12,4720,0.015094
9,12,5546,0.102733
